In [125]:
using JuMP
using Gurobi
using DataStructures: OrderedDict
using Iterators: subsets

In [101]:
function verify_form(model::Model)
    isempty(model.obj.aff.vars) &&
    isempty(model.sdpconstr) && 
    isempty(model.quadconstr) && 
    isempty(model.conicconstrDuals) &&
    all(c.lb == -Inf || c.ub == Inf for c in model.linconstr)
end

verify_form (generic function with 1 method)

In [102]:
function get_objective_matrix(model::Model, vars, params)
    @assert verify_form(model)
    H = zeros(Float64, (length(vars), length(vars)))
    for i in 1:length(model.obj.qcoeffs)
        v1 = model.obj.qvars1[i]
        v2 = model.obj.qvars2[i]
        c = model.obj.qcoeffs[i]
        @assert !haskey(params, v1)
        @assert !haskey(params, v2)
        H[vars[v1], vars[v2]] = c
    end
    H
end

function get_linconstr_matrices(model, vars, params)
    @assert verify_form(model)
    G = zeros(Float64, (length(model.linconstr), length(vars)))
    W = zeros(Float64, length(model.linconstr))
    S = zeros(Float64, (length(model.linconstr), length(params)))
    
    for (i, constraint) in enumerate(model.linconstr)
        for (j, var) in enumerate(constraint.terms.vars)
            coeff = constraint.terms.coeffs[j]
            if constraint.lb == -Inf
                W[i] = constraint.ub
                if haskey(vars, var)
                    G[i, vars[var]] = coeff
                else
                    S[i, params[var]] = -coeff
                end
            else
                @assert constraint.ub == Inf
                W[i] = -constraint.lb
                if haskey(vars, var)
                    G[i, vars[var]] = -coeff
                else
                    S[i, params[var]] = coeff
                end
            end
        end
    end
    G, W, S
end

get_linconstr_matrices (generic function with 1 method)

In [103]:
model = Model(solver=GurobiSolver())
@variables model begin
    x
    y
end

vars = OrderedDict{Variable, Int}(zip([x, y], 1:2))

@variables model begin
    p[1:5] == 0
end

params = OrderedDict{Variable, Int}(zip(p, 1:length(p)))

constraints = [
    @constraint model x <= 1 + p[1]
    @constraint model x >= -4 - p[2]
    @constraint model y <= 0 + p[3]
    @constraint model y >= -3 - p[4]
    @constraint model x + 2y <= -4 + p[5]
]

@objective model Min (x)^2 + (y)^2

@assert verify_form(model)

model

Minimization problem with:
 * 5 linear constraints
 * 7 variables
Solver is Gurobi

In [104]:
get_linconstr_matrices(model, vars, params)

(
[1.0 0.0; -1.0 0.0; … ; 0.0 -1.0; 1.0 2.0],

[1.0,4.0,-0.0,3.0,-4.0],
[1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0])

In [105]:
get_objective_matrix(model, vars, params)

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

In [106]:
result = solve(model)

Optimize a model with 5 rows, 7 columns and 11 nonzeros
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 4 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.96535824e+05 -5.11497999e+05  1.50e+03 2.98e+02  6.82e+05     0s
   1   8.23980246e+01 -1.18898855e+04  1.73e+01 4.12e+00  1.17e+04     0s
   2   1.67502711e+01 -7.27792459e+03  0.00e+00 4.12e-06  1.46e+03     0s
   3   1.67239697e+01 -2.75129759e+01  0.00e+00 2.

:Optimal

In [108]:
?JuMP.ConstraintRef

No documentation found.

**Summary:**

```
immutable JuMP.ConstraintRef{M<:JuMP.AbstractModel,T<:JuMP.AbstractConstraint} <: Any
```

**Fields:**

```
m   :: M<:JuMP.AbstractModel
idx :: Int64
```


In [113]:
function get_duals(model::Model)
    @assert verify_form(model)
    model.linconstrDuals
end

get_duals (generic function with 1 method)

In [114]:
get_duals(model)

5-element Array{Float64,1}:
 -0.0
  0.0
 -0.0
  0.0
 -1.6

In [115]:
function get_active_set(model::Model)
    get_duals(model) .== 0
end

get_active_set (generic function with 1 method)

In [116]:
get_active_set(model)

5-element BitArray{1}:
  true
  true
  true
  true
 false

In [117]:
G, W, S = get_linconstr_matrices(model, vars, params)

(
[1.0 0.0; -1.0 0.0; … ; 0.0 -1.0; 1.0 2.0],

[1.0,4.0,-0.0,3.0,-4.0],
[1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0])

In [120]:
lambda = get_duals(model)

5-element Array{Float64,1}:
 -0.0
  0.0
 -0.0
  0.0
 -1.6

In [121]:
A = get_active_set(model)

5-element BitArray{1}:
  true
  true
  true
  true
 false

In [158]:
function linearly_independent_subset(G, active_indices)
    num_constraints = min(length(active_indices), size(G, 2))
    for Ai in subsets(active_indices, num_constraints)
        GA = @view G[Ai,:]
        if rank(GA) == num_constraints
            return Ai
        end
    end
    error("No linearly indepdendent set of rows could be found")
end

linearly_independent_subset (generic function with 1 method)

In [185]:
function critical_region(model, vars, params)
    H = get_objective_matrix(model, vars, params)
    G, W, S = get_linconstr_matrices(model, vars, params)
    active_constraint_indices = [i for (i, constr) in enumerate(model.linconstr) if isapprox(getvalue(constr.terms), constr.ub) || isapprox(getvalue(constr.terms), constr.lb)]
    @show active_constraint_indices
    
    Ai = linearly_independent_subset(G, active_constraint_indices)
    GA = @view G[Ai,:]
    WA = @view W[Ai,:]
    SA = @view S[Ai,:]   
    @show H GA
    x = collect(keys(params))
    lambdaA = -inv(GA * inv(H) * GA') * (WA + SA * x)
    z = vec(-inv(H) * GA' * lambdaA)
    @assert isapprox(getvalue.(z), getvalue.(collect(keys(vars))))
    
    @show G S W z x
    @show G * z .- S * x
    
    region_constraints = vcat(G * z .- S * x .- W,
        -lambdaA)
end
    
    
    

critical_region (generic function with 1 method)

In [186]:
critical_region(model, vars, params)

active_constraint_indices = [5]
H = [1.0 0.0; 0.0 1.0]
GA = [1.0 2.0]
G = [1.0 0.0; -1.0 0.0; 0.0 1.0; 0.0 -1.0; 1.0 2.0]
S = [1.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 1.0]
W = [1.0,4.0,-0.0,3.0,-4.0]
z = JuMP.GenericAffExpr{Float64,JuMP.Variable}[0.2 p[5] - 0.8,0.4 p[5] - 1.6]
x = JuMP.Variable[p[1],p[2],p[3],p[4],p[5]]
G * z .- S * x = JuMP.GenericAffExpr{Float64,JuMP.Variable}[0.2 p[5] - p[1] - 0.8,-0.2 p[5] - p[2] + 0.8,0.4 p[5] - p[3] - 1.6,-0.4 p[5] - p[4] + 1.6,-4]


6×1 Array{JuMP.GenericAffExpr{Float64,JuMP.Variable},2}:
 0.2 p[5] - p[1] - 1.8 
 -0.2 p[5] - p[2] - 3.2
 0.4 p[5] - p[3] - 1.6 
 -0.4 p[5] - p[4] - 1.4
 0                     
 0.2 p[5] - 0.8        

In [166]:
getvalue.(collect(keys(vars)))

2-element Array{Float64,1}:
 -0.8
 -1.6

In [135]:
G[[1,2],:]

2×2 Array{Float64,2}:
  1.0  0.0
 -1.0  0.0

In [122]:
GA = G[A,:]
WA = W[A,:]
SA = S[A,:]
lambdaA = lambda[A]

4-element Array{Float64,1}:
 -0.0
  0.0
 -0.0
  0.0

In [123]:
GA

4×2 Array{Float64,2}:
  1.0   0.0
 -1.0   0.0
  0.0   1.0
  0.0  -1.0

In [124]:
rank(GA)

2

In [107]:
lambda = getdual.(model.linconstr)

LoadError: MethodError: no method matching getdual(::JuMP.GenericRangeConstraint{JuMP.GenericAffExpr{Float64,JuMP.Variable}})[0m
Closest candidates are:
  getdual([1m[31m::JuMP.ConstraintRef{JuMP.Model,JuMP.GenericRangeConstraint{JuMP.NonlinearExprData}}[0m) at /home/rdeits/.julia/v0.5/JuMP/src/nlp.jl:54
  getdual([1m[31m::JuMP.ConstraintRef{JuMP.Model,JuMP.GenericSOCConstraint{JuMP.GenericNormExpr{2,Float64,JuMP.Variable}}}[0m) at /home/rdeits/.julia/v0.5/JuMP/src/JuMP.jl:587
  getdual([1m[31m::JuMP.ConstraintRef{JuMP.Model,JuMP.GenericRangeConstraint{JuMP.GenericAffExpr{Float64,JuMP.Variable}}}[0m) at /home/rdeits/.julia/v0.5/JuMP/src/JuMP.jl:545
  ...[0m

In [25]:
getdual(constraints[5])

-1.600000000804784

In [26]:
(getvalue(x), getvalue(y))

(3.200000000180182,1.3999999993969696)

In [29]:
typeof(constraints[1])

JuMP.ConstraintRef{JuMP.Model,JuMP.GenericRangeConstraint{JuMP.GenericAffExpr{Float64,JuMP.Variable}}}